In [1]:
import math
from sklearn.utils import resample
import scipy as sp, numpy as np
import scipy.io as io
import pandas as pd
from sklearn.preprocessing import scale, StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import LeaveOneGroupOut, LeaveOneOut
import matplotlib.pyplot as plt
%matplotlib inline

In [86]:
#Reading Table S4 from paper into Y (18 week 12 groups).
s4=pd.read_csv("full data.csv", names = ['Control1','Control2', 'Control3','LAB + EtOH1','LAB + EtOH2','LAB + EtOH3','Acid + Yeast1','Acid + Yeast2','Acid + Yeast3','Acid + EtOH1','Acid + EtOH2','Acid + EtOH3'])
#Reading Table S1SS2S3 into X (0 week mean 1 group)
s123=pd.read_csv("s1s2s3X.csv", names = ['Week0Mean','SD','SourceTable'])

In [165]:
#Y= array shape (131, 12)
Y=s4.to_numpy()
Xmean=s123.iloc[:,0].to_numpy()
Xsd=s123.iloc[:,1].to_numpy()
#create 3 week0 samples from the mean and sd, assume normal dist
np.random.seed(1)
X=np.empty([Xmean.size,3])
for i in range (Xmean.size):
    X[i]=np.random.normal(Xmean[i],Xsd[i], 3)
#replace nan (constituents not found in week0 with tiny value to prevent PLSR ValueError: Input contains NaN)
X=np.nan_to_num(X, nan=1e-99)
# X array shape 131,3

In [160]:
#separate Y into each exp grouping, transpose so rows are test groups/cols are variable constituents
YControl=s4.iloc[:,0:3].T.to_numpy()
YLabEtoh=s4.iloc[:,3:6].T.to_numpy()
YAcidYeast=s4.iloc[:,6:9].T.to_numpy()
YAcidEtoh=s4.iloc[:,-3:len(s4.columns)].T.to_numpy()
Y=np.append([YControl, YLabEtoh,YAcidYeast],[YAcidEtoh], axis=0)
Y[0]
#Y[0][0] is exp grp 1 of YControl, 131 constit for 1 grp

0.1854

In [163]:
Xmean=s123.iloc[:,0].to_numpy()
Xsd=s123.iloc[:,1].to_numpy()
#create 3 week0 samples from the mean and sd, assume normal dist
np.random.seed(1)
X=np.empty([Xmean.size,3])
for i in range (Xmean.size):
    X[i]=np.random.normal(Xmean[i],Xsd[i], 3)
#replace nan (constituents not found in week0 with tiny value to prevent PLSR ValueError: Input contains NaN)
X=np.nan_to_num(X, nan=1e-99).T
X.shape

(3, 131)

In [159]:
scoresXY=PLSR2.fit_transform(X,YControl)

In [143]:
scoresXY[1][:,1]

array([-5.31871037,  1.82659935,  3.49211102])

In [153]:
labels=['YControl','YLabEtoh','YAcidYeast','YAcidEtoh']
numPC=2
PLSR2 = PLSRegression(n_components=numPC)
scoresXY=PLSR2.fit_transform(X,Y)
r2y=PLSR2.score(X, Y)
print('r2y: ',r2y)
for i in range(4):
    plt.scatter(scoresXY[1][:,0],scoresXY[1][:,1],label=labels[0])
    print(PLSR2.y_loadings_)

scoresXY=PLSR2.fit_transform(X,YLabEtoh)
r2y=PLSR2.score(X, YLabEtoh)
print('r2y: ',r2y,' for YLabEtoh')
plt.scatter(scoresXY[1][:,0],scoresXY[1][:,1],label=labels[1])

scoresXY=PLSR2.fit_transform(X,YAcidYeast)
r2y=PLSR2.score(X, YAcidYeast)
print('r2y: ',r2y,' for YAcidYeast')
plt.scatter(scoresXY[1][:,0],scoresXY[1][:,1],label=labels[2])

scoresXY=PLSR2.fit_transform(X,YAcidEtoh)
r2y=PLSR2.score(X, YAcidEtoh)
print('r2y: ',r2y,' for YAcidEtoh')
plt.scatter(scoresXY[1][:,0],scoresXY[1][:,1],label=labels[3])

plt.legend()
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [131, 3]

In [121]:
PLSR2.y_loadings_
# scoresArray[1] for 2 PCs. [1][][0] for x scores. [1][][1] for y scores
# plot 12 total scores/exp groups, 3 for each exp group
for pairs in range(0,12):
    if pairs%3==0:
        c=c+1
        plt.scatter(scoresXY[1][pairs][0],scoresArray[1][0][pairs][1], color=colors[c],label=labels[c])
    else:
        plt.scatter(scoresXY[1][pairs][1],scoresArray[1][0][pairs][1], color=colors[c])
plt.title('PLSR Scores Fig 2A')
plt.legend()
plt.ylabel('PC 2')
plt.xlabel('PC 1')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.show()

array([[ 0.18107685,  0.04617431],
       [ 0.18443103,  0.02247773],
       [ 0.08862726, -0.1876655 ],
       [ 0.18508842, -0.01352441],
       [-0.17876243,  0.0568962 ],
       [ 0.18508191,  0.01364212],
       [ 0.03076136, -0.21067898],
       [-0.16090997,  0.10622523],
       [ 0.18274523,  0.03642272],
       [ 0.15761561,  0.11258872],
       [-0.16813715, -0.09015652],
       [ 0.18508633,  0.01356221],
       [ 0.18246722,  0.0382275 ],
       [ 0.17315083,  0.0765352 ],
       [ 0.1562819 , -0.11502933],
       [-0.14684358, -0.13049196],
       [ 0.17386319, -0.07436139],
       [ 0.18269029, -0.03678661],
       [ 0.17811407, -0.05953329],
       [ 0.13053088,  0.15176379],
       [ 0.17518329, -0.07012978],
       [ 0.15396133, -0.11910951],
       [ 0.18432437, -0.02361019],
       [ 0.15414846, -0.1187879 ],
       [ 0.13740155, -0.1434904 ],
       [ 0.18134   ,  0.04478296],
       [-0.14144119, -0.13818388],
       [ 0.18250177,  0.03800804],
       [ 0.18491596,

In [92]:
numPC=2
scoresArray=np.empty([numPC-1,2],dtype=object)
PLSR2 = PLSRegression(n_components=numPC)
scoresXY=PLSR2.fit_transform(X,Y)
scoresArray[0][0]=scoresXY[0]
scoresArray[0][1]=scoresXY[1]

r2y=PLSR2.score(X, Y)
print('r2y: ',r2y,' for number PCs=',numPC)

plt.scatter(scoresXY[0],scoresXY[1])

ValueError: Found input variables with inconsistent numbers of samples: [3, 12]

In [89]:
scoresXY[1].shape

(131, 2)

In [91]:
PLSR2.y_loadings_.shape

(12, 2)